In [2]:
!pip install requests beautifulsoup4


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import re
import requests
from bs4 import BeautifulSoup


In [4]:
url = "https://elpais.com/mexico/"

In [5]:
# Obtener contenido de la página
html_content = requests.get(url)
html_content.text[:1000]

'<!DOCTYPE html><html lang="es-MX"><head><meta charSet="UTF-8"/><meta name="viewport" content="width=device-width, initial-scale=1.0"/><link rel="preconnect" href="//static.elpais.com"/><link rel="preconnect" href="//assets.adobedtm.com"/><link rel="preconnect" href="//sdk.privacy-center.org"/><link rel="preload" href="https://imagenes.elpais.com/resizer/v2/ELVL57XNDJAATHPAKFGBGIX4Z4.jpg?auth=58d833165969f4deff6b41c4bf9d36430e042e9bbe663d79e264dc8fd987aa83&amp;width=414&amp;height=233&amp;smart=true" imageSrcSet="https://imagenes.elpais.com/resizer/v2/ELVL57XNDJAATHPAKFGBGIX4Z4.jpg?auth=58d833165969f4deff6b41c4bf9d36430e042e9bbe663d79e264dc8fd987aa83&amp;width=414&amp;height=233&amp;smart=true 414w,https://imagenes.elpais.com/resizer/v2/ELVL57XNDJAATHPAKFGBGIX4Z4.jpg?auth=58d833165969f4deff6b41c4bf9d36430e042e9bbe663d79e264dc8fd987aa83&amp;width=828&amp;height=466&amp;smart=true 828w,https://imagenes.elpais.com/resizer/v2/ELVL57XNDJAATHPAKFGBGIX4Z4.jpg?auth=58d833165969f4deff6b41c4bf9d

In [6]:
# Convertimos la respuesta a un elemento de BeautifulSoup
soup = BeautifulSoup(html_content.text, "html.parser")

In [7]:
# Obtener los principales articulos
articles = soup.find_all("article", class_ = ["c", "c-d"])
articles[:10]

[<article class="c c-d c--m" data-word="870"><div class="c_m"><span class="c_m_c _pr _db" href="https://elpais.com/mexico/2025-09-19/la-trama-de-hernan-bermudez-pone-contra-las-cuerdas-a-adan-augusto-lopez-y-mide-la-politica-anticorrupcion-de-sheinbaum.html?autoplay=1" title="La trama de Bermúdez arrincona a Adán Augusto López y mide la política anticorrupción de Sheinbaum"><video class="_re" data-poster="https://imagenes.elpais.com/resizer/v2/ELVL57XNDJAATHPAKFGBGIX4Z4.jpg?auth=58d833165969f4deff6b41c4bf9d36430e042e9bbe663d79e264dc8fd987aa83&amp;width=414&amp;height=233&amp;smart=true" data-src="https://vdmedia.elpais.com/elpaistop/multimedia/20259/19/1090995.mp4" data-title="La trama de Bermúdez arrincona a Adán Augusto López y mide la política anticorrupción de Sheinbaum" height="315" id="vloop_601" loop="" muted="" width="560"><source data-src="https://vdmedia.elpais.com/elpaistop/multimedia/20259/19/1090995.mp4" type="video/mp4"/></video></span></div><header class="c_h"><h2 class=

In [8]:
"""
Creación de elementos para cada artículo
- Titulo
- Enlace
- Autor
- Descripción
"""

articles_parsed = []

for a in articles:
    a_title_tag = a.find("h2")
    a_title = a_title_tag.get_text(strip = True) if a_title_tag else None

    a_link_tag = a_title_tag.find("a")
    a_link = a_link_tag["href"] if a_link_tag else None

    a_auth_tag = a.find(class_ = "c_a_a")
    a_auth = a_auth_tag.get_text(strip = True) if a_auth_tag else None

    a_desc_tag = a.find(class_ = "c_d")
    a_desc = a_desc_tag.get_text(strip = True) if a_desc_tag else None

    article_content = {
        "titulo": a_title, 
        "enlace": a_link, 
        "autor": a_auth,
        "descripcion": a_desc
    }
    
    articles_parsed.append(article_content)

In [11]:
articles_parsed[:3]

[{'titulo': 'La trama de Bermúdez arrincona a Adán Augusto López y mide la política anticorrupción de Sheinbaum',
  'enlace': 'https://elpais.com/mexico/2025-09-19/la-trama-de-hernan-bermudez-pone-contra-las-cuerdas-a-adan-augusto-lopez-y-mide-la-politica-anticorrupcion-de-sheinbaum.html',
  'autor': 'Pablo Ferri',
  'descripcion': 'Los procesos en contra del exjefe policial de Tabasco apuntan a la figura del exgobernador, que lo nombró para el puesto'},
 {'titulo': 'Una tarjeta de crédito, un sobrino y una luz encendida: las claves de la detención de Hernán Bermúdez en Paraguay',
  'enlace': 'https://elpais.com/mexico/2025-09-19/una-tarjeta-de-credito-un-sobrino-y-una-luz-encendida-las-claves-de-la-detencion-de-hernan-bermudez-en-paraguay.html',
  'autor': 'Federico Rivas Molina',
  'descripcion': None},
 {'titulo': 'El exjuez Arturo Zaldívar defiende la enmienda al amparo y deja pistas de una dedicatoria que apunta a Salinas Pliego',
  'enlace': 'https://elpais.com/mexico/2025-09-19/

In [12]:
# Buscar articulos de violencia o problemas sociales
violencia_re = r'violencia|agresion(nes)?|golpe(s)?|muerto(s)?|muerte(s)?|corrupcion|crueldad|discriminacion|probreza|delincuencia|desempleo|marginacion'

matched_articles = []

for a in articles_parsed:
    a_title_lower = a["titulo"].lower()
    a_title = re.sub(r"á", "a", a_title_lower)
    a_title = re.sub(r"é", "e", a_title)
    a_title = re.sub(r"í", "i", a_title)
    a_title = re.sub(r"ó", "o", a_title)
    a_title = re.sub(r"ú", "u", a_title)
    a_title = re.sub(r"[^\w\s]", "", a_title)

    if re.search(violencia_re, a_title):
        matched_articles.append(a)

In [13]:
matched_articles[:5]

[{'titulo': 'La trama de Bermúdez arrincona a Adán Augusto López y mide la política anticorrupción de Sheinbaum',
  'enlace': 'https://elpais.com/mexico/2025-09-19/la-trama-de-hernan-bermudez-pone-contra-las-cuerdas-a-adan-augusto-lopez-y-mide-la-politica-anticorrupcion-de-sheinbaum.html',
  'autor': 'Pablo Ferri',
  'descripcion': 'Los procesos en contra del exjefe policial de Tabasco apuntan a la figura del exgobernador, que lo nombró para el puesto'},
 {'titulo': 'La tragedia de Iztapalapa escala a 25 muertos con cuatro más en 12 horas',
  'enlace': 'https://elpais.com/mexico/2025-09-19/la-tragedia-de-iztapalapa-escala-a-25-muertos-con-cuatro-mas-en-12-horas.html',
  'autor': 'Verónica M. Garrido',
  'descripcion': None},
 {'titulo': 'Israel afirma que golpeará Ciudad de Gaza con una fuerza “sin precedentes”',
  'enlace': 'https://elpais.com/internacional/2025-09-19/israel-cierra-la-segunda-via-de-salida-de-ciudad-de-gaza-y-anuncia-una-ofensiva-sin-precedentes-sobre-la-zona.html',
 